## Lexical Analysis

In [ ]:
import pandas as pd
import os
train_fold = 'train/'
test_fold = 'test/'
ext = '.txt'
for filename in os.listdir(test_fold):
    if filename.endswith('.txt'):
        path = test_fold + filename
        ffile = 'ntest/' + filename
        os.system('clang -cc1 -dump-tokens ' + path + ' 2> ' + ffile)

## Feature

In [ ]:
'''
usle = set(['', '1', '_Bool', 'eof', 'r_pare', 'r_paren', 'r_brace', 'r_square', 'semi', 
            'train/0ae1/8c126b029441457e.txt:81:17:', 'train/e892/a1eb2cf347244c59.txt:22:33:'])
feature = list(token - usle)
feature
'''

In [ ]:
# store
with open('./feature.pkl', 'wb') as f:
    pickle.dump(feature,f)

In [ ]:
# load
import pickle
def load_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data
feature = load_data('feature.pkl')

## Token Frequency

In [ ]:
def vectorize(path):
    with open(path) as f:
        docs = f.readlines()
        for i in range(len(docs)):
            p = docs[i].find('\t')
            docs[i] = docs[i][:p].split(' ')[0]
        d = Counter(docs)
        v = []
        for i in feature:
            v.append(d[i])
    return v

In [ ]:
import pandas as pd
import os
from collections import Counter
train_fold = 'nntrain/'
test_fold = 'test/'
ext = '.txt'
test = pd.read_csv('sample_submission.csv')
predictions = []



vvv = []
token = set([])
for clone_fold in os.listdir(train_fold):
    if len(clone_fold) == 4:
        clone = os.listdir(train_fold + clone_fold)
        vv = []
        for filename in clone:
            if filename.endswith('.txt'):
                path = train_fold + clone_fold + '/' + filename
                vv.append(vectorize(path))
        vvv.append(vv)

In [ ]:
len(vvv[0][0])

In [ ]:
with open('./TrainVec.pkl', 'wb') as f:
    pickle.dump(vvv, f)

## Dataset

In [ ]:
import random
X = []
y = []
X_ = []
y_ = []
# train
for i in range(83):
    for j in range(2000):
        a = random.randint(0,499)
        b = random.randint(0,499)
        if a != b:
            v = vvv[i][a] + vvv[i][b]
            X.append([v])
            y.append(1)
for i in range(83*20000):
    a = random.randint(0,82)
    b = random.randint(0,82)
    if a != b:
        z1 = random.randint(0,499)
        z2 = random.randint(0,499)
        v = vvv[a][z1] + vvv[b][z2]
        X.append([v])
        y.append(0)
# valid
for i in range(73, 83):
    for j in range(1000):
        a = random.randint(0,499)
        b = random.randint(0,499)
        if a != b:
            v = vvv[i][a] + vvv[i][b]
            X_.append([v])
            y_.append(1)
for i in range(10*10000):
    a = random.randint(73,82)
    b = random.randint(73,82)
    if a != b:
        z1 = random.randint(0,499)
        z2 = random.randint(0,499)
        v = vvv[a][z1] + vvv[b][z2]
        X_.append([v])
        y_.append(0)

In [ ]:
import numpy as np
X = np.array(X).reshape(-1,2,73)
y = np.array(y)
X_ = np.array(X_).reshape(-1,2,73)
y_ = np.array(y_)

In [ ]:
from sklearn.utils import shuffle
# shuffle
X, y = shuffle(X, y, random_state=0)
X_, y_ = shuffle(X_, y_, random_state=0)
X.shape, y.shape, X_.shape, y_.shape

In [ ]:
d = {'X': X, 'y': y}
d_ = {'X_': X_, 'y_': y_}

In [ ]:
# store training set and validation set
import pickle
with open('./MyData1.pkl', 'wb') as f:
    pickle.dump(d, f)
with open('./MyData2.pkl', 'wb') as f:
    pickle.dump(d_, f)

## Testset

In [ ]:
# test
pair = [i.split('_') for i in test.id1_id2]
path_pair = [['ntest/' + a + ext, 'ntest/' + b + ext] for a,b in pair]

In [ ]:
vec_pair = [[vectorize(a), vectorize(b)] for a,b in path_pair]

In [ ]:
vec = np.array([np.array(i).reshape(1,2,73) for i in vec_pair])

In [ ]:
with open('./TestVec.pkl', 'wb') as f:
    pickle.dump(vec,f)